<a href="https://colab.research.google.com/github/Jungyoonlim/middlemarch/blob/main/Middlemarch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **Middlemarch Project.**

Objective: Break down George Eliot's Middlemarch into relevant story metadata.

In [31]:
from google.colab import files

uploaded = files.upload()

Saving middlemarch.txt to middlemarch (3).txt


In [32]:
file_path = 'middlemarch.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    middlemarch_text = file.read()

print("Text loaded successfully.")


Text loaded successfully.


In [33]:
import re
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
nltk.download('punkt')

# Assuming 'middlemarch_text' contains the entire text of Middlemarch

# Step 1: Remove Gutenberg Headers and Footers
# The headers/footers vary, but often include phrases like "Project Gutenberg"
start_marker = "PRELUDE."
end_marker = "THE END"

start = middlemarch_text.find(start_marker)
end = middlemarch_text.find(end_marker)

# Extract only the main text
main_text = middlemarch_text[start:end]

# Step 2: Tokenization
# Split into sentences
sentences = sent_tokenize(main_text)

# Split into words
words = word_tokenize(main_text)

print(f"Number of sentences: {len(sentences)}")
print(f"Number of words: {len(words)}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Number of sentences: 11900
Number of words: 370620


Named Entity Recognition

In [34]:
import spacy

nlp = spacy.load("en_core_web_sm")
nlp.max_length = len(main_text)
doc = nlp(main_text)

character_entities = set()

for entity in doc.ents:
  if entity.label_ == 'PERSON':
    if not any(char.isdigit() for char in entity.text):
      character_entities.add(entity.text.strip())

for character in character_entities:
  print(character)

I. ‘He’s
BEN JONSON
Clara
Harfager
l’aube de la
Lucifer
Thou
James
Freshitt Hall
Love
Henry VIII
Cadwallader—“when
Augustine
Spilkins
Wright
Dissenting hymn-books
ALFRED DE MUSSET
Willoughby
Dover
Vicar
Gospel
Isaiah
Majesty
Solomon Featherstone
Faulkner
Miss Winifred
Puritanic
Galen
Lowick Parish
Madonna
Edwin Larcher
Chairman
Peg
Burke
Blessed Virgin
Brooke
Le Légataire
Si
Der
Murder
Diamond
Adam
Hiram Ford’s
Africay
Lady Blessington
Johnson
Adolf Naumann
Lord Megatherium
Scott
Miss
Brooke
Theresa
Nicholas
Bulstrode
Cam
S.
Apocalypse
Rosy
a Servile Crawler
Crabbe
Brookes
Rambler
Hanged
Monk
riddle?—it
Call Fred Vincy
Miss Noble
Fred
Vincy
Wimple
Blucher
Mr Casaubon
Dantzic
Camden Farebrother
Fred felt
Enmity
brio
Jeersteen
Overreach
jamais rien
L’homme qui rit
Nimrod
Truberry
Ballard
emphatically,—“she
Rosamond
Vincy
George
Fuggon
Dante
Lady Chettam
Paracelsus
Fatigue
Nicholas Bulstrode
Dodo
Luck
Whate’er
Shallow
Southey
Hiram Ford
Lowick Gate
Churchyard Lane
Blindman
Ode to Duty
Hou

In [35]:
from textblob import TextBlob

sentence = TextBlob(sentences[0])
print(sentence.sentiment)

Sentiment(polarity=0.0, subjectivity=0.0)


Summary Generation

In [36]:
!pip install sumy

Topic Modeling

Text Summarization

In [37]:
import nltk

In [38]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.lsa import LsaSummarizer

num_sentences_summary = 10

parser = PlaintextParser.from_string(main_text, Tokenizer("english"))
summarizer_list=("TextRankSummarizer:","LexRankSummarizer:","LuhnSummarizer:","LsaSummarizer") #list of summarizers
summarizers = [TextRankSummarizer(), LexRankSummarizer(), LuhnSummarizer(), LsaSummarizer()]

for i, summarizer in enumerate(summarizers):
  print(summarizer_list[i])
  for sentence in summarizer(parser.document, num_sentences_summary):
    print(sentence)
  print("-" * 30)


TextRankSummarizer:


KeyboardInterrupt: ignored

Topic Modeling

- Preprocess and Tokenize the Text
- Create Dictionary and Corpus
- LDA for Topic Modeling

In [39]:
processed_text = preprocess(main_text)

print(processed_text[:10])

NameError: ignored

In [ ]:
import re
# Adjust the regular expression to match the chapter pattern exactly
chapter_pattern = r'\nCHAPTER [IVXLCDM]+\.'  # This includes the newline character and period

# Use re.split to split the text into chapters
chapters = re.split(chapter_pattern, main_text)

chapters = chapters[1:]

# Filter out any empty strings that might have been created during the split
chapters = [chapter.strip() for chapter in chapters if chapter.strip()]

print(f"Total chapters found: {len(chapters)}")
for i in range(3):
    print(f"Start of chapter {i+1}: {chapters[i][:100]}")  # Preview first 100 characters

In [ ]:
from gensim.corpora import Dictionary

dictionary = Dictionary([processed_text])

dictionary.filter_extremes(no_below=5, no_above=0.7, keep_n=50000)

corpus = [dictionary.doc2bow(text) for text in [processed_text]]


Fine-Tune Models
- Narrative Generation and Understanding
- Custom NER Model


In [40]:
pip install transformers

In [41]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("fill-mask", model="bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [42]:
fill_mask_pipe = pipeline("fill-mask", model="bert-base-uncased")

# Example usage
masked_text = "The main character in Middlemarch is [MASK]."
predictions = fill_mask_pipe(masked_text)

for prediction in predictions:
    print(prediction)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'score': 0.037618719041347504, 'token': 4300, 'token_str': 'arthur', 'sequence': 'the main character in middlemarch is arthur.'}
{'score': 0.013670222833752632, 'token': 2848, 'token_str': 'peter', 'sequence': 'the main character in middlemarch is peter.'}
{'score': 0.013061015866696835, 'token': 2852, 'token_str': 'dr', 'sequence': 'the main character in middlemarch is dr.'}
{'score': 0.011167364194989204, 'token': 17001, 'token_str': 'sgt', 'sequence': 'the main character in middlemarch is sgt.'}
{'score': 0.009783466346561909, 'token': 2198, 'token_str': 'john', 'sequence': 'the main character in middlemarch is john.'}


In [43]:
middlemarch_paragraphs = middlemarch_text.split('\n\n')  # Assuming two newlines separate paragraphs

In [44]:
from transformers import BartTokenizer
from datasets import Dataset

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

def process_data_to_model_inputs(batch):
    # Tokenize the inputs and labels
    inputs = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=512)
    return inputs

# Create a dataset object
dataset = Dataset.from_dict({'text': middlemarch_paragraphs})
dataset = dataset.map(process_data_to_model_inputs, batched=True)

Map:   0%|          | 0/4944 [00:00<?, ? examples/s]

In [46]:
!pip install accelerate -U
!pip install transformers[torch]

In [48]:
!pip install transformers[torch] -U

In [49]:
!pip install torch -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 955.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [47]:
from transformers import BartForConditionalGeneration, Trainer, TrainingArguments

model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer
)

trainer.train()


ImportError: ignored

In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model="./fine_tuned_bart", tokenizer=tokenizer)

generated_text = generator("Example prompt", max_length=50)
print(generated_text[0]['generated_text'])

Further Metadata Extraction
- Character Relationships
- Plot Analysis
- Style and Tone Analysis